In [1]:
%cd ../
import os
import sys
import time
import requests
import pandas as pd
from scipy import signal
from scipy import fftpack

import numpy as np
from matplotlib import pyplot as plt
from stix.core import stix_datatypes as sdt
from stix.core import mongo_db as db
from stix.core import stix_datetime
%matplotlib notebook
mdb=db.MongoDB()
SPID = 54118

/home/xiaohl/FHNW/STIX/gsw/STIX_python
[INFO 2020-09-11 17:18:37] : IDB loaded from stix/data/idb/idb.sqlite


## Data preparation

In [2]:


def get_lc_data(file_id):
    # query database and retrieve the packets
    
    packets = mdb.select_packets_by_run(file_id, SPID)
    if not packets:
        print(f'No QL LC packets found for run {file_id}')
        return None, None
    else:
        print('Number of  LC packets found for run {}: {}'.format(file_id,packets.count()))

    lightcurves = {}
    unix_time = []
    for pkt in packets:
        packet = sdt.Packet(pkt)
        if not packet.isa(SPID):
            continue
        #fig = None

        scet_coarse = packet[1].raw
        scet_fine = packet[2].raw
        start_scet = scet_coarse + scet_fine / 65536.

        int_duration = (packet[3].raw + 1) * 0.1


        num_lc_points = packet.get('NIX00270/NIX00271')[0]
        lc = packet.get('NIX00270/NIX00271/*.eng')[0]
        for i in range(len(lc)):
            if i not in lightcurves:
                lightcurves[i] = []
            lightcurves[i].extend(lc[i])
        unix_time.extend([
            stix_datetime.scet2unix(start_scet + x * int_duration)
            for x in range(num_lc_points[0])
        ])
    return lightcurves, unix_time



In [3]:
lcs,t=get_lc_data(268)
lightcurve=np.array(lcs[0]) #4-10 keV
unix_time=np.array(t)
mean_counts=np.mean(lightcurve)
std_counts=np.std(lightcurve)
print(mean_counts,std_counts)


Number of  LC packets found for run 268: 103


<ipython-input-2-e40630fb4b07>:9: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print('Number of  LC packets found for run {}: {}'.format(file_id,packets.count()))


265.88530297957817 67.0933804157653


# Peak detection algorithm

In [4]:
#peak detection algorithm
def find_edges(series, peak_xpos):
    mean=np.mean(series)
    
    peak_heights=series[peak_xpos]
    for pi in peak_xpos:
        ymax=peak_heights[pi]
        ylevel = (ymax-mean)*0.1+ mean 
        
        
        




def find_peaks(data,peak_width=10, distance=75):
    mean_counts=np.mean(data)
    std_counts=20 #np.std(data)
    height=mean_counts+std_counts
    prominence=std_counts
    #find local maximums which satisfy the conditions
    
    
    xpeak, properties = signal.find_peaks(data, 
                                          height=height,
                                         # threshold=threshold,
                                          prominence=prominence,
                                                width=peak_width,
                                                distance=distance
                                         )
    print('Number of detected peaks:', xpeak.size)
    print('Properties:')
    print(properties)
    



    if xpeak.size>0:
        peak_values=data[xpeak]
        left_bases=properties['left_bases'].astype(int)
        right_bases=properties['right_bases'].astype(int)
      
        peak_unix_times = unix_time[xpeak]
        peaks_utc=[stix_datetime.unix2utc(x) for x in peak_unix_times]
        print('Peaks:')
        #print(results_full)

        fig1=plt.figure()

        plt.plot(unix_time, data)
        #plt.yscale('log')
        plt.vlines(peak_unix_times, 0.8*peak_values, 1.5*peak_values,color='C1')
        plt.hlines(peak_values, unix_time[left_bases], unix_time[right_bases], color='darkblue' )
        plt.vlines(unix_time[left_bases], 0.5*peak_values, 1.5*peak_values,color='darkblue')
        plt.vlines(unix_time[right_bases], 0.5*peak_values, 1.5*peak_values,color='darkblue')
        
        plt.show()

      
        return peak_unix_times, peak_values


#t, v=find_peaks(lightcurve)



## Low-pass filter

In [5]:
# Low pass filter to fiter high frequence pattern caused  by integer compression, noise etc.

Wn=0.03 #cut off frequency    

order=4 #butterworth filter order
b,a=signal.butter(order, Wn, 'low', analog=False)#analog 
w, h = signal.freqs(b, a)
fig=plt.figure()
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [1 / second]')
#plt.ylabel('Amplitude [dB]')
plt.grid(which='both', axis='both')
#plt.axvline(Wn, color='green') # cutoff frequency
plt.show()
    
#y = signal.filtfilt(b, a, data)




<IPython.core.display.Javascript object>

In [6]:
lc_smoothed = signal.filtfilt(b, a, lightcurve)
fig=plt.figure()
plt.plot(unix_time,lightcurve,'r', label='orignal')
plt.plot(unix_time,lc_smoothed,'b', label='smoothed')
plt.yscale('log')
plt.legend(loc='upper left')
plt.show()

<IPython.core.display.Javascript object>

In [7]:
times, peak_values= find_peaks(lc_smoothed)

Number of detected peaks: 26
Properties:
{'peak_heights': array([ 288.34951181,  323.27891183,  327.8667716 ,  298.26313229,
        301.10585862,  319.59967229,  592.38539707,  581.63466894,
        332.44251593,  290.90445588,  383.50607547,  295.35999941,
        304.42857918,  303.59686997,  306.70068771,  326.73233595,
        533.11601088,  294.51012287,  287.15023598,  287.78274029,
        455.27314356,  295.90060376,  322.50920613, 2439.76573928,
        308.92753522,  292.43755542]), 'prominences': array([  34.74338258,   69.49904929,   75.29951947,   40.00695758,
         43.13733405,   44.97877946,  341.81407664,   76.46549899,
         71.832831  ,   28.40822103,  128.4340506 ,   30.37951421,
         49.33833183,   29.29783501,   51.98096873,   72.26075713,
        285.46446364,   28.17327113,   34.02960163,   35.32219684,
        205.33376534,   41.28597572,   68.36152929, 2189.75124017,
         57.81414768,   37.56301136]), 'left_bases': array([11922, 11922, 11922, 171

NameError: name 'results_full' is not defined

fig3=plt.figure()
plt.plot(unix_time, lightcurve)
plt.vlines(times, 0.8*peak_values, 1.5*peak_values,color='C1')
plt.show()

width=75
order=2
lc_savgol=signal.savgol_filter(lightcurve, width, order)

fig4=plt.figure()
plt.plot(unix_time, lightcurve,'b', label='original')
plt.plot(unix_time,lc_savgol,'r',label='smoothed')
#plt.plot(unix_time,lc_smoothed,'g',label='smoothed 2')
plt.legend('upper right')
plt.show()

times_savgol, peak_values_savgol= find_peaks(lc_savgol)

fig=plt.figure()
plt.plot(times,peak_values,'x')
plt.vlines(times_savgol, 0.8*peak_values_savgol,peak_values_savgol*1.2, color='C1')
plt.show()

In [ ]:
np.std(lightcurve[0:500])